#   ROS2 | Exercise 4 - Computer Vision

##  Task 3 - Object Detection

### Pre-requisites
Before you begin this Notebook you have to complete the following tasks explained in the handout
- [x] Setting up the gazebo simulation as a ROS package
- [x] Setting up Conda environment to run this jupyter notebook

### Introduction
Object detection and its importance in robotics. Opencv Intro. Some images

There are 3 parts in this Notebook which covers the following
   - 3.1 : Basic Image processing and Opencv contour detection
   - 3.2 : Object detection and Tracking with Video
   - 3.3 : Object detection and Tracking with ROS2

***
### 3.1 Basic Image processing with OpenCV 

### 3.1.1
Familiarize yourself with the basic functionalities available in openCV for basic image reading and visualization with this reference [link](https://docs.opencv.org/4.x/db/deb/tutorial_display_image.html). It has both C++ & Python code. We are following python in this notebook. Execute the following code that reads an image file and displays it in an opencv window.

In [1]:
import imutils
import cv2
import numpy as np

frame = cv2.imread('image.png')
cv2.imshow("Tracking",frame)
cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

### 3.1.2

It is important to  pre-process raw input images in computer vision tasks such as object detection and tracking. OpenCV imgproc module contains several functionalities which are often used in image processing.

Convertion between color spaces is necessary to generate image masks. Execute and observe the following code where you convert an image from RGB to HSV color space. The code generates a mask of the green color object.

**Your task**

- [ ] Change HSV values in order to generate masks of other objects.

**References**
- [Concept of HSL and HSV Color spaces](https://en.wikipedia.org/wiki/HSL_and_HSV)
- [Opencv basic Image processing functionalities](https://docs.opencv.org/4.x/d7/da8/tutorial_table_of_content_imgproc.html)
- [OpenCV RGB-HSV color converstions documentation](https://docs.opencv.org/3.4.6/de/d25/imgproc_color_conversions.html#color_convert_rgb_hsv)


In [2]:
# lower and upper boundaries for HSV thresholds
green_l = (45, 86, 6)
green_u = (64, 255, 255)

blue_l = (90, 50, 70)
blue_u = (128, 255, 255)

red_l = (0, 100, 20)
red_u = (10, 255, 255)

yellow_l = (20, 100, 100)
yellow_u = (30, 255, 255)

purple_l = (145, 255, 120)
purple_u = (155, 255, 255)


def prep(frame):
    frame = imutils.resize(frame, width=600)
    ## Todo 3.1.3 gaussian blur
    blurred = cv2.GaussianBlur(frame,(11,11),0.5)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv,green_l,green_u)
    mask2 = cv2.inRange(hsv,blue_l,blue_u)
    mask3 = cv2.inRange(hsv,red_l,red_u)
    mask4 = cv2.inRange(hsv,yellow_l,yellow_u)
    mask5 = cv2.inRange(hsv,purple_l,purple_u)
    
    mask = mask1 + mask2 + mask3 + mask4 + mask5
    
    erosion_element  = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    dilation_element = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    
    ## Todo 3.1.3 gaussian blur
    mask = cv2.erode(mask, erosion_element)
    mask = cv2.dilate(mask,dilation_element) 
    return mask, frame


mask,frame = prep(frame)

cv2.imshow("Image",frame)
cv2.imshow("Masked Image",mask)
cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

### 3.1.3

The above implementation provides a satisfactory result. However, when dealing with noisy images it is important to perform morphological transformations such as smoothing, erotion and dialation to get better results.

**Your task**

Read the following documentations and implement the following transformations inside the prep_frame() function above.


- [ ] Apply Gaussian blur with a kernel size of 11x11
- [ ] Apply erosion
- [ ] Apply dialation

**References**

- [Smoothing Images with OpenCV](https://docs.opencv.org/4.5.2/d4/d13/tutorial_py_filtering.html)
- [Perform Erosion and Dialation on the image mask using OpenCV](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html)

### 3.1.4

Finding contours in an image is useful in object detection and tracking. The code snippet below extracts contours from the image file and draws a circle with its center as the same center of detected contour. Execute the code below and observe the results.

**References**
- [ Detecting contours with OpenCV](https://docs.opencv.org/4.5.0/d4/d73/tutorial_py_contours_begin.html)

In [3]:
### Find contours and display
def find_cnts(mask):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

cnts = find_cnts(mask)

min_circles = []
bounding_rects = []
rotated_rects = []
hulls = []
for contour in cnts:
    ((x, y), radius) = cv2.minEnclosingCircle(contour) ## A different contour?
    min_circles.append((x,y,radius))
    
    x,y,w,h = cv2.boundingRect(contour) ## A different contour?
    bounding_rects.append((x,y,w,h))
    
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)    
    rotated_rects.append(box)
    
    hull = cv2.convexHull(contour)
    hulls.append(hull)


frame1 = frame
for circle, rect, box, hull in zip(min_circles,bounding_rects,rotated_rects, hulls):
    x,y,radius = circle
    x1,y1,w,h = rect
    
    cv2.circle(frame1, (int(x), int(y)), int(radius),(0, 0, 255), 2)
    cv2.rectangle(frame1,(x1,y1),(x1+w,y1+h),(0,255,0),2)
    cv2.drawContours(frame1,[box],0,(0,0,255),2)
    cv2.drawContours(frame1,[hull],0,(0,255,0),2)

    
cv2.imshow("Circle", frame1)


## ToDo 3.1.5 
#  1. Bounding rectancgle
#  2. Convex Hull


cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

### 3.1.5
The above implementation only uses a minimum enclosing circle for labelling the detected image. There are more labelling options such as bounding boxes and convex hull which may come handy in various computer vision tasks.

**Your task**

- [ ] Implement a bounding rectangle and rotated rectangle on the detected contours by modifying the code in 3.1.4 
- [ ] Implement a convex hull on the detected contours by modifying the code in 3.1.4

**References**
- [Contour features in OpenCV](https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html)
- [Creating Bounding boxes and circles for contours](https://docs.opencv.org/4.x/da/d0c/tutorial_bounding_rects_circles.html)
-[Convex Hull](https://docs.opencv.org/4.x/d7/d1d/tutorial_hull.html)

***
### 3.2 Object detection and Tracking with Video

### 3.2.1

Execute and observe the below code snippet which reads frames from a given video file at a pre-defined frame rate.

**Your task**
- [ ] Implement pre-processing and find contour of the moving blob on by modifiying the same code below
- [ ] Implement a suitable labelling mode (Enclosed Circle / bounding Box or convex hull) and display the resulted tracking on a seperated window

**Tip** : Use the same functions from 3.1. Avoid creating duplicate functions and unnecessarily lenghty code

In [4]:
from imutils.video import VideoStream
import time

vs = cv2.VideoCapture("test_video.avi")
time.sleep(2.0)

frame_rate = 30
prev = 0

while True :
    time_elapsed = time.time() - prev

    
    if time_elapsed > 1./frame_rate:
        frame = vs.read()
        frame = frame[1]
        if frame is None:
            break
        mask, frame = prep(frame)
        cnts = find_cnts(mask)
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        prev = time.time()
            
cv2.destroyAllWindows()   

---

###  3.3 Object detection and Tracking with ROS2

### 3.3.1 

In applications concerning robotic perception, so often visual data are obtained via camera sensors interfaced with ros. In this section you are provided with a ros-ignition simulation package which publish camera sensor data. The code snippet below subscribes to the camera sensor data and stream on to an OpenCV window. 

Assuming you have successfully build the package on your system using the instructions from handout, Open a New shell in you terminal and launch the simulation.

```
$ source /opt/ros/foxy/setup.bash
$ cd ~/ws
$ source install/setup.bash
$ ros2 launch walking_actor cam_world.launch.py
```

While the simulation is running in background , run the code snippet below and observe the result

- To close the cv2 window , select the window and press 'q'
- To stop the execution of code snippet, double press on 'i' in keyboard or interrupt the kernel from menu


**Your task**

- [ ] Modify the python code to track the walking actor. You can use any feature of the actor to track.
- [ ] Display the tracking in a seperate cv2 window


**References**
- [ROS2 python subscriber/publisher](https://docs.ros.org/en/foxy/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Publisher-And-Subscriber.html)
- [rclpy API Documentation](https://docs.ros2.org/foxy/api/rclpy/api/execution_and_callbacks.html#module-rclpy.executors)


In [6]:
import sys
import rclpy
from rclpy.node import Node
from cv_bridge import CvBridge
import numpy as np
import cv2
from sensor_msgs.msg import Image, CameraInfo
#print(sys.path)

bridge = CvBridge()

class Get_Images(Node):

    def __init__(self):
        # Initialize the node
        super().__init__('Image_Subscriber')
        # Initialize the subscriber
        self.subscription_ = self.create_subscription( Image,'/camera', self.listener_callback,10)
        self.subscription_  # prevent unused variable warning
        timer_period = 0.1  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.K = True

    def listener_callback(self, msg):
        height = msg.height
        width = msg.width
        channel = msg.step//msg.width
        #frame = np.reshape(msg.data, (height, width, channel))
        self.frame =  bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        self.get_logger().info("Image Received")
        return self.frame
    
    def timer_callback(self):
        if self.K == True:
            mask, frame1 = prep(self.frame)
            cnts = find_cnts(mask)
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.imshow("Tracking",frame1)
            #cv2.imshow("Tracking",self.frame)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                self.get_logger().info('Closing stream window..')
                self.K = False
                cv2.destroyAllWindows()
        
                
    def stop_stream(self):
        self.get_logger().info('Stopping the stream ...')
        

try:
    #rclpy.shutdown()
    rclpy.init(args=None)
    image_subscriber = Get_Images()
    rclpy.spin(image_subscriber)

except KeyboardInterrupt:
    # executes on keyboard kernal interrupt with double pressing button "i"
    image_subscriber.stop_stream()
    image_subscriber.destroy_node()
    rclpy.shutdown()

[INFO] [1670860186.624956215] [Image_Subscriber]: Image Received
[INFO] [1670860186.663803095] [Image_Subscriber]: Image Received
[INFO] [1670860186.675349273] [Image_Subscriber]: Image Received
[INFO] [1670860186.689332096] [Image_Subscriber]: Image Received
[INFO] [1670860186.745821752] [Image_Subscriber]: Image Received
[INFO] [1670860186.780673330] [Image_Subscriber]: Image Received
[INFO] [1670860186.818987618] [Image_Subscriber]: Image Received
[INFO] [1670860186.873107635] [Image_Subscriber]: Image Received
[INFO] [1670860186.922895058] [Image_Subscriber]: Image Received
[INFO] [1670860186.970805257] [Image_Subscriber]: Image Received
[INFO] [1670860187.023386867] [Image_Subscriber]: Image Received
[INFO] [1670860187.067195448] [Image_Subscriber]: Image Received
[INFO] [1670860187.128934931] [Image_Subscriber]: Image Received
[INFO] [1670860187.170695413] [Image_Subscriber]: Image Received
[INFO] [1670860187.236089623] [Image_Subscriber]: Image Received
[INFO] [1670860187.274354

[INFO] [1670860192.867647084] [Image_Subscriber]: Image Received
[INFO] [1670860192.923327763] [Image_Subscriber]: Image Received
[INFO] [1670860192.964960609] [Image_Subscriber]: Image Received
[INFO] [1670860193.013767391] [Image_Subscriber]: Image Received
[INFO] [1670860193.064577251] [Image_Subscriber]: Image Received
[INFO] [1670860193.106492037] [Image_Subscriber]: Image Received
[INFO] [1670860193.163318519] [Image_Subscriber]: Image Received
[INFO] [1670860193.212403692] [Image_Subscriber]: Image Received
[INFO] [1670860193.250454201] [Image_Subscriber]: Image Received
[INFO] [1670860193.305113481] [Image_Subscriber]: Image Received
[INFO] [1670860193.342985109] [Image_Subscriber]: Image Received
[INFO] [1670860193.411030617] [Image_Subscriber]: Image Received
[INFO] [1670860193.451068776] [Image_Subscriber]: Image Received
[INFO] [1670860193.490515996] [Image_Subscriber]: Image Received
[INFO] [1670860193.545999535] [Image_Subscriber]: Image Received
[INFO] [1670860193.582676

[INFO] [1670860198.900321215] [Image_Subscriber]: Image Received
[INFO] [1670860198.943195589] [Image_Subscriber]: Image Received
[INFO] [1670860198.987270435] [Image_Subscriber]: Image Received
[INFO] [1670860199.040908837] [Image_Subscriber]: Image Received
[INFO] [1670860199.092627223] [Image_Subscriber]: Image Received
[INFO] [1670860199.132925838] [Image_Subscriber]: Image Received
[INFO] [1670860199.186092955] [Image_Subscriber]: Image Received
[INFO] [1670860199.232534034] [Image_Subscriber]: Image Received
[INFO] [1670860199.282643814] [Image_Subscriber]: Image Received
[INFO] [1670860199.332869108] [Image_Subscriber]: Image Received
[INFO] [1670860199.377042486] [Image_Subscriber]: Image Received
[INFO] [1670860199.427698755] [Image_Subscriber]: Image Received
[INFO] [1670860199.474645986] [Image_Subscriber]: Image Received
[INFO] [1670860199.524518649] [Image_Subscriber]: Image Received
[INFO] [1670860199.572902328] [Image_Subscriber]: Image Received
[INFO] [1670860199.617074